
















































































# **Big Five Personalities Test Clusterization**

The Big Five personality traits, also known as the five-factor model (FFM) and the OCEAN model, is a taxonomy, or grouping, for personality traits. When factor analysis (a statistical technique) is applied to personality survey data, some words used to describe aspects of personality are often applied to the same person. For example, someone described as conscientious is more likely to be described as "always prepared" rather than "messy". This theory is based therefore on the association between words but not on neuropsychological experiments. This theory uses descriptors of common language and therefore suggests five broad dimensions commonly used to describe the human personality and psyche.

The theory identifies five factors:

- `OPENNESS TO EXPERIENCE` (inventive/curious vs. consistent/cautious)
- `CONSCIENTIOUSNESS` (efficient/organized vs. extravagant/careless)
- `EXTRAVERSION` (outgoing/energetic vs. solitary/reserved)
- `AGREEABLENESS` (friendly/compassionate vs. critical/rational)
- `NEUROTICISM` (sensitive/nervous vs. resilient/confident)

(Source: Wikipedia)



Download do arquivo no Kaggle no link: [Big Five Personality Test](https://www.kaggle.com/tunguz/big-five-personality-test)

# 1: Importando Bibliotecas


In [ ]:
# para manipulação de dados
import pandas as pd
import seaborn as sns
import numpy as np
from datetime import datetime
from pandas.tseries.offsets import Day, Week, MonthEnd
from datetime import datetime
from datetime import timedelta
from google.cloud import bigquery
import os
from io import open
pd.options.display.max_columns = 150

# para visualização de dados
import matplotlib.pyplot as plt
%matplotlib inline
import plotly
import plotly.express as px
import plotly.offline as py
plotly.offline.init_notebook_mode()
import plotly.graph_objs as go
import plotly.io as pio                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
import pytz

# 2: Acessando Fontes de Dados

* This dataset contains 1,015,342 questionnaire answers collected online by Open Psychometrics.

In [ ]:
# Salva os dados dos testes em um DataFrame
data_big_five = pd.read_csv('../input/big-five-personality-test/IPIP-FFM-data-8Nov2018/data-final.csv', sep='\t')

In [ ]:
data_big_five.head()

# 3: Manipulando os datasets

Para este problema, usaremos apenas as features correspondentes às 50 perguntas:

In [ ]:
# removendo colunas que não são as perguntas do questionário
data = data_big_five.drop(data_big_five.columns[50:110], axis=1, inplace=False)
data.head()

Tratamento de dados missing e/ou incoerentes

In [ ]:
# estatística descritiva dos dados
pd.options.display.float_format = "{:.2f}".format
data.describe()

- Verifica-se que o valor mínimo registrado para as questões é 0 (zero), o que é incoerente com a pontuação do teste (escala 1 a 5)

- Esses valores precisam ser identificados e removidos

In [ ]:
# verificando quantidade de registros 0 (zero) em cada coluna
data[data == 0].count()

In [ ]:
data = data[data > 0.00]

In [ ]:
# conferindo quantidade de registros 0 (zero) em cada coluna após remoção
data[data == 0].count()

In [ ]:
# verificando quantidade de registros NULL em cada coluna
data.isna().sum()

In [ ]:
# removendo os registros NULL do dataset
data = data.dropna()

In [ ]:
# conferindo quantidade de registros NULL em cada coluna após remoção
data.isna().sum()

# 4: Análise Exploratória



Utilizando o Pandas Profiling

In [ ]:
# instalando o pandas profiling
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

In [ ]:
# importando o ProfileReport
import pandas_profiling
from pandas_profiling import ProfileReport\

In [ ]:
# executando o ProfileReport
# For large datasets, there is a configuration that disables expensive computations (such as correlations and dynamic binning)
profile = ProfileReport(data, title='Relatório - Pandas Profiling', html={'style':{'full_width':True}}, minimal = True)

In [ ]:
# salvando o relatório no disco
profile.to_file(output_file="Relatorio_ProfileReport.html")

# 5: Criação de Modelos

### Otimização de grupos para clusterização

- Ao deparar com um problema de agrupamento de dados, a primeira pergunta que vem em mente é: em quantos grupos queremos agrupar os dados?
- Em primeiro lugar, é necessário entender a regra de negócio.
- Como o teste foi construído em cima da hipótese de que existem 5 tipos de perdonalidades diferentes, é natural que 5 seja o número de grupos diferentes com características que mais se destacam e que o algoritmo é capaz de identificar.
- Entretanto, existem métodos estatísticos que estimam o número ideal de clusters baseados na variabilidade dos dados, como o Visualizador KElbow da biblioteca Yellowbrick, que será usado para fins de confirmação.

Biblioteca yellowbrick

- Facilita a visualização de modelos e ajustes na visualização exploratória de modelos

In [ ]:
# instalando a biblioteca yellowbrik
!pip install -U yellowbrick

Método KElbowVisualizer

- Vai ajudar a olhar graficamente como um modelo se comportaria com cada número de clusters diferentes para identificar o valor ideal, baseado na variabilidade dos dados.
- Faz uma análise da variância entre as observações para cada K.

In [ ]:
# importando o método KElbowVisualizer e o método de clusterização K-Means
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

In [ ]:
# criando objeto do tipo KMeand e KElbowVisualizer e definindo um range de valores de K de 2 a 10 para comparação
kmeans = KMeans()
visualizer = KElbowVisualizer(kmeans, k=(2,10))

In [ ]:
# selecionando uma amostra aleatória dos dados com 20000 observações
data_sample = data.sample(n=20000, random_state=42)

In [ ]:
# executando o teste
visualizer.fit(data_sample)        # Fit the data to the visualizer
visualizer.show()                  # Finalize and render the figure

O resultado acima mostra que o valor ideal de clusters para o problema é k = 5.

### K-MEANS 

- K-means clustering is a type of Unsupervised Learning, which is used when you have unlabeled data (i.e., data without defined categories or groups). The goal of this algorithm is to find groups in the data, with the number of groups represented by the variable K.

In [ ]:
# carrega novamente o pacote KMeans
from sklearn.cluster import KMeans

In [ ]:
# cria o objeto do tipo KMeans com o parâmetro de clusters igual a 5
kmeans = KMeans(n_clusters=5)

In [ ]:
# treina o algoritmo
k_fit = kmeans.fit(data)

In [ ]:
# calcula os rótulos dos clusters para cada registro
predicoes = k_fit.labels_

In [ ]:
# atribui os resultados em uma nova coluna no dataframe
data['clusters'] = predicoes
data.head()

# 6: Resultados

Distribuição dos dados em cada grupo

In [ ]:
# quantidade de registros atribuída a cada cluster
data["clusters"].value_counts()

In [ ]:
# utilizando Plotly Express
fig = px.bar(data, y = data["clusters"].value_counts().index, x = data["clusters"].value_counts(),orientation='h',
             labels={"x": "Quantidade de registros",
                     "y": "Clusters"})
fig.show()

In [ ]:
# Agrupando os registros por grupos
data.groupby('clusters').mean()

Cálculo da média de cada grupo de questões para verificar um padrão

In [ ]:
# definindo listas para os conjuntos de colunas de cada tipo de personalidade
col_list = list(data)
ext = col_list[0:10]
est = col_list[10:20]
agr = col_list[20:30]
csn = col_list[30:40]
opn = col_list[40:50]

In [ ]:
# calculando a média dos valores das colunas de cada tipo de personalidade
data_soma = pd.DataFrame()
data_soma['EXTRAVERSION'] = data[ext].sum(axis=1)/10
data_soma['NEUROTICISM'] = data[est].sum(axis=1)/10
data_soma['AGREEABLENESS'] = data[agr].sum(axis=1)/10
data_soma['CONSCIENTIOUSNESS'] = data[csn].sum(axis=1)/10
data_soma['OPENNESS'] = data[opn].sum(axis=1)/10
data_soma['clusters'] = predicoes

In [ ]:
# visualizando as médias por grupo
data_soma

In [ ]:
# visualizando a distribuição dos clusters dentro dos conjuntos de personalidades
fig = px.scatter_matrix(data_soma, dimensions= ['EXTRAVERSION', 'NEUROTICISM', 'AGREEABLENESS', 'CONSCIENTIOUSNESS', 'OPENNESS'], color="clusters")
fig.show()

In [ ]:
# calculando os valores médios dos registros para cada tipo de personalidade e visualizando por cluster
data_clusters = data_soma.groupby('clusters').mean()
data_clusters

In [ ]:
# visualizando as médias de valores dos conjuntos de personalidades em cada cluster
data_clusters.plot()

# 7: Deploy

- Será utilizada a biblioteca Gradio para publicar uma interface para rodar o modelo e coletar novos dados para uso no modelo.
- A interface do Gradio pede uma função para a qual o Gradio vai passar os inputs para que seja feito o processamento (nesse caso, para que seja consumido o modelo de clusterização)

In [ ]:
# instalando a biblioteca Gradio
!pip install gradio

In [ ]:
# fazendo o import da biblioteca
import gradio as gr

Lendo os dados com as questões do arquivo codebook.txt

In [ ]:
# o arquivo contém mais dados do que apenas as questões, por isso filtram-se as 50 linhas a partir da 7ª linha
codebook = open('../input/big-five-personality-test/IPIP-FFM-data-8Nov2018/codebook.txt').read().split("\n")
dicio_questions = codebook[7:57]    # apenas o conteúdo das perguntas

In [ ]:
# verificando os dados das questões
dicio_questions

Separando apenas as questões dos códigos das questões

In [ ]:
#retorna apenas o que está após os caracteres "\t" e insere em uma lista
questions = []
for q in dicio_questions:
  q = str(q)   # garante que é string
  questions.append(q[q.find("\t"):].lstrip())

In [ ]:
questions

Criando os inputs dinâmicos para o Gradio

In [ ]:
# cria lista que vai receber os dados inputados através da interface do Gradio e
# cria objeto no formato de slider com pontuação mínima de 1 e máxima de 5
inputs_questions = []
for q in questions:
  obj_input = gr.inputs.Slider(minimum=1,maximum=5,step=1,default=3,label=q)
  inputs_questions.append(obj_input)

In [ ]:
# verificando a criação dos inputs
inputs_questions

Criando a interface no Gradio e a função *cluster_personalities*

In [ ]:
def cluster_personalities(*outputs_questions):
    outputs_questions = np.array(outputs_questions).reshape(1, -1)
    return k_fit.predict(outputs_questions)

iface = gr.Interface(
                    fn = cluster_personalities,
                    title = "Big Five Personality Test",
                    description = "Teste para detecção de traços de personalidade",
                    inputs = inputs_questions,
                    outputs="text")
iface.launch(share=True)